In [14]:
import warnings

# with warnings.catch_warnings():
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")
from sklearn import preprocessing

from functions.process_fopdt import ProcessModel
from functions.mpc_nn import Mpc_nn

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

from pickle import dump, load
from sklearn.preprocessing import MinMaxScaler
import time



# For LSTM model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.models import load_model

import tclab

# Load NN model parameters and MinMaxScaler
model_params = load(open('model_param_MIMO.pkl', 'rb'))
s1 = model_params['Xscale']
s2 = model_params['yscale']
window = model_params['window']

# Load NN models (onestep prediction models)
model_lstm_one = load_model('MPC_MIMO_TCLab_onestep_LSTM.h5')
model_trans_one = load_model('MPC_MIMO_TCLab_onestep_Trans.h5')

# Load NN models (multistep prediction models)
model_lstm_multi = load_model('MPC_MIMO_TCLab_multistep_LSTM.h5')
model_trans_multi = load_model('MPC_MIMO_TCLab_multistep_Trans.h5')

# # FOPDT Parameters
# K=1.0      # gain
# tau=2.0    # time constant
ns = 15*60  # Simulation Length
t = np.linspace(0, ns, ns + 1)
delta_t = t[1] - t[0]

nu = 2
ny = 2

# Define horizons
P = 5  # Prediction Horizon
M = 4  # Control Horizon

# Input Sequence
u = np.zeros((ns + 1, nu))
# u[5:,0] = .3
# u[5:,1] = .6

## Set initial temp - both T1 and T2??
T1_set = 35
T2_set = 30

# Setpoint Sequence

sp1 = np.zeros(ns + 1)
sp1 = sp1+T1_set
# sp1[80:] = 1.5

sp2 = np.zeros(ns + 1)
sp2 = sp2+T2_set
# sp2[80:] = 1.5

sp = np.array([sp1, sp2]).T
target = np.array([T1_set,T2_set]).T

# Controller setting
maxmove = 1

# Process simulation
yp = np.zeros((ns + 1, ny))
yp_nn = np.zeros((ns + 1, ny, P))








# p = ProcessModel(delta_t)
m = Mpc_nn(window*60, nu, ny, P, M, sp1, sp2, multistep=1, model_one=model_trans_one, model_multi=model_trans_multi)
# multistep = 0 : sequential onestep prediction MPC
# multistep = 1 : simultaneous multistep prediction MPC


print(window)








5


In [2]:

# the control interval in seconds
CI = 30

T1_arr = np.zeros((P*60))
T2_arr = np.zeros((P*60))
Q1_arr = np.zeros((P*60))
Q2_arr = np.zeros((P*60))





tsim = 15*60 # time in sec




## Get initial data
#uhat = np.zeros((M, nu))



lab = tclab.TCLab()


for i in range(len(T1_arr)):
    t_in = time.time()
    T1_arr[i] = lab.T1
    T2_arr[i] = lab.T2
    Q1_arr[i] = lab.Q1()
    Q2_arr[i] = lab.Q2()
    if(T1_arr[i] < T1_set):
        lab.Q1(80)
    else:
        lab.Q1(0)
    if(T2_arr[i] < T2_set):
        lab.Q2(80)
    else:
        lab.Q2(0)
        
    print(T1_arr[i], Q1_arr[i])
    

    t_out = time.time()
    time.sleep(1-(t_out-t_in))

TCLab version 0.4.9
Arduino Leonardo connected on port COM5 at 115200 baud.
TCLab Firmware 2.0.1 Arduino Leonardo/Micro.
70.85 0.0
70.85 0.0
70.85 0.0
70.85 0.0
70.85 0.0
71.075 0.0
71.14 0.0
71.107 0.0
70.882 0.0
70.85 0.0
70.527 0.0
70.527 0.0
70.527 0.0
70.205 0.0
70.205 0.0
69.883 0.0
69.689 0.0
69.561 0.0
69.238 0.0
69.238 0.0
68.916 0.0
68.787 0.0
68.594 0.0
68.594 0.0
68.271 0.0
68.207 0.0
67.949 0.0
67.627 0.0
67.305 0.0
66.982 0.0
66.66 0.0
66.66 0.0
66.338 0.0
66.016 0.0
65.822 0.0
65.693 0.0
65.371 0.0
65.049 0.0
64.727 0.0
64.404 0.0
64.082 0.0
63.76 0.0
63.438 0.0
63.115 0.0
62.793 0.0
62.471 0.0
62.148 0.0
62.116 0.0
61.826 0.0
61.504 0.0
61.182 0.0
61.182 0.0
60.859 0.0
60.859 0.0
60.537 0.0
60.537 0.0
60.215 0.0
59.957 0.0
59.893 0.0
59.57 0.0
59.57 0.0
59.248 0.0
59.248 0.0
58.926 0.0
58.604 0.0
58.313 0.0
58.249 0.0
57.959 0.0
57.637 0.0
57.637 0.0
57.314 0.0
56.992 0.0
56.831 0.0
56.67 0.0
56.348 0.0
56.025 0.0
55.929 0.0
55.703 0.0
55.381 0.0
55.381 0.0
55.059 0.0
5

In [16]:
uhat = np.zeros((M,nu))
for i in range(tsim):
    t_in = time.time()
    T1 = lab.T1
    T2 = lab.T2
    Q1 = lab.Q1()
    Q2 = lab.Q2()

    T1_arr = np.append(T1_arr,T1)
    T2_arr = np.append(T2_arr,T2)
    Q1_arr = np.append(Q1_arr,Q1)
    Q2_arr = np.append(Q2_arr,Q2)

    u = np.vstack((Q1_arr,Q2_arr)).T
    y = np.vstack((T1_arr,T2_arr)).T
    target = np.array([T1_set,T2_set]).T
    
    
    if i%30 == 0:
        print(u)
        print(y)
        u_window = u[-60*P::30]
        y_window = y[-60*P::30]
        
        
        #uhat = m.run(uhat, u_window, y_window, target)
        uhat = m.run(uhat, u_window, y_window, sp[i])

        lab.Q1(uhat[0][0])
        lab.Q2(uhat[1][0])

    t_out = time.time()
    time.sleep(1-(t_out-t_in))



# plt.plot(t, yp)
plt.subplot(2, 1, 1)
plt.plot(t, y[:, 0])
plt.plot(t, y[:, 1])
# plt.plot(t, yp_nn[:, :, 0], '-.')
plt.step(t, sp[:, 0])
plt.step(t, sp[:, 1])
plt.subplot(2, 1, 2)
plt.step(t, u[:, 0])
plt.step(t, u[:, 1])
plt.show()

print("break point")

[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]

AttributeError: 'numpy.ndarray' object has no attribute 'transform'